# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_6 MWT', 'verschil_TUG',
               'verschil_BMI', 'verschil_Gewicht',
               'verschil_Lenigheid', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_Conditie'])
y = df['verschil_Conditie']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['kPersonenMetLaagsteInkomen', 'AfstandTotKinderdagverblijf', 'AfstandTotConsultatiebureau', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotBioscoop', 'AfstandTotSauna', 'BeperkingInHoren']
Dropped features: ['AfstandTotCafetariaED', 'ErvarenGezondheidGoedZeerGoed', 'AfstandTotRestaurant', 'OvermatigDrinker', 'AfstandTotHuisartsenpraktijk', 'GeweldsEnSeksueleMisdrijven', 'BouwjaarVanaf2000', 'ErnstigOvergewichtObesitas', 'AfstandTotPoppodium', 'GemiddeldeWoningwaarde', 'PersonenautoSNaarOppervlakte', 'EigendomOnbekend', 'AfstandTotBibliotheek', 'AfstandTotVolkstuin', 'BeperkingInZien', 'AfstandTotAttractie', 'AfstandTotOvDagelLevensmiddelen', 'UrenMantelzorgPerWeek', 'PercentageOnbewoond', 'PersonenautoSOverigeBrandstof', 'InBezitWoningcorporatie', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotKunstijsbaan', 'AfstandTotSportterrein', 'PersonenautoSPerHuishouden', '

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------
| 1         | -0.0322   | 9.672     | 0.08727   | 1.222     |
| 2         | -0.03219  | 6.215     | 0.05407   | 1.196     |
| 3         | -0.03217  | 3.526     | 0.01261   | 1.799     |
| 4         | -0.03216  | 1.996     | 0.08047   | 1.107     |
| 5         | -0.03219  | 6.398     | 0.03826   | 1.956     |
| 6         | -0.03218  | 4.875     | 0.0986    | 3.117     |
| 7         | -0.03196  | 3.49      | 0.08538   | 2.063     |
| 8         | -0.03217  | 2.23      | 0.02202   | 1.869     |
| 9         | -0.0322   | 7.209     | 0.01775   | 1.001     |
| 10        | -0.0322   | 7.639     | 0.06598   | 1.178     |
| 11        | -0.0322   | 9.592     | 0.05043   | 1.886     |
| 12        | -0.03197  | 3.508     | 0.08171   | 2.088     |
| 13        | -0.03197  | 3.427     | 0.08291   | 2.149     |
| 14        | -0.0319   | 3.2       | 0.006563  | 2.051     |
| 15    

SVR(C=3.1128489082554966, gamma=0.0012711768740856556)

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotApotheek']
Dropped features: ['AfstandTotCafetariaED', 'ErvarenGezondheidGoedZeerGoed', 'BeperkingInHoren', 'AfstandTotKinderdagverblijf', 'AfstandTotBioscoop', 'AfstandTotRestaurant', 'OvermatigDrinker', 'AfstandTotHuisartsenpraktijk', 'GeweldsEnSeksueleMisdrijven', 'BouwjaarVanaf2000', 'ErnstigOvergewichtObesitas', 'AfstandTotPoppodium', 'GemiddeldeWoningwaarde', 'PersonenautoSNaarOppervlakte', 'EigendomOnbekend', 'AfstandTotBibliotheek', 'AfstandTotVolkstuin', 'BeperkingInZien', 'AfstandTotAttractie', 'AfstandTotOvDagelLevensmiddelen', 'UrenMantelzorgPerWeek', 'PercentageOnbewoond', 'PersonenautoSOverigeBrandstof', 'AfstandTotBuitenschoolseOpvang', 'InBezitWoningcorporatie', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotKunstijsbaan', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotSportterrein', 'PersonenautoSPerHuishouden', 'AfstandTotConsultatiebureau', 'AfstandTotPodiumkunstenTotaal', 'AfstandTotOpenbaarGroenTotaal', 'kPersonenMetLaagsteInkomen', 'Af

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -0.03152  | 8.369     | 0.2351    | 9.726     | 133.1     |
| 2         | -0.03151  | 6.229     | 0.5278    | 9.963     | 340.4     |
| 3         | -0.03124  | 2.988     | 0.3545    | 5.181     | 565.0     |
| 4         | -0.03152  | 17.74     | 0.7925    | 10.94     | 740.6     |
| 5         | -0.03152  | 8.032     | 0.7366    | 11.1      | 709.4     |
| 6         | -0.03152  | 8.713     | 0.3794    | 6.004     | 158.2     |
| 7         | -0.03118  | 1.719     | 0.693     | 3.493     | 972.1     |
| 8         | -0.03152  | 19.62     | 0.5868    | 11.46     | 923.6     |
| 9         | -0.03151  | 5.634     | 0.209     | 8.071     | 665.2     |
| 10        | -0.03134  | 4.276     | 0.8614    | 6.497     | 470.0     |
| 11        | -0.03118  | 1.575     | 0.6832    | 2.166     | 999.9     |
| 12        | -0.03152  | 19.78     | 

RandomForestRegressor(bootstrap=False, max_depth=2,
                      max_features=0.6930430664086109, min_samples_split=3,
                      n_estimators=972)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['AfstandTotHotelED', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotConsultatiebureau', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotVolkstuin', 'AfstandTotOpenDroogNatTerrein', 'InBezitWoningcorporatie', 'AfstandTotOpenNatNatuurlijkTerrein']
Dropped features: ['AfstandTotCafetariaED', 'ErvarenGezondheidGoedZeerGoed', 'BeperkingInHoren', 'AfstandTotKinderdagverblijf', 'AfstandTotBioscoop', 'AfstandTotRestaurant', 'OvermatigDrinker', 'AfstandTotHuisartsenpraktijk', 'GeweldsEnSeksueleMisdrijven', 'BouwjaarVanaf2000', 'ErnstigOvergewichtObesitas', 'AfstandTotPoppodium', 'GemiddeldeWoningwaarde', 'PersonenautoSNaarOppervlakte', 'EigendomOnbekend', 'AfstandTotBibliotheek', 'BeperkingInZien', 'AfstandTotAttractie', 'AfstandTotOvDagelLevensmiddelen', 'UrenMantelzorgPerWeek', 'PercentageOnbewoond', 'PersonenautoSOverigeBrandstof', 'AfstandTotBuitenschoolseOpvang', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotKunstijsbaan', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotSp

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -0.1768   | 0.09271   | 43.38     | 0.01064   | 507.4     |
| 2         | -0.1785   | 0.02523   | 13.24     | 0.07231   | 413.3     |
| 3         | -0.1761   | 0.04387   | 32.65     | 0.04615   | 921.6     |
| 4         | -0.1761   | 0.09405   | 70.31     | 0.06619   | 134.4     |
| 5         | -0.1765   | 0.07256   | 92.73     | 0.0287    | 650.2     |
| 6         | -0.1761   | 0.02226   | 67.07     | 0.09335   | 731.1     |
| 7         | -0.176    | 0.01689   | 58.72     | 0.04637   | 154.8     |
| 8         | -0.1767   | 0.0382    | 60.74     | 0.07811   | 175.4     |
| 9         | -0.1763   | 0.04839   | 27.73     | 0.05866   | 552.8     |
| 10        | -0.1765   | 0.03707   | 83.5      | 0.01485   | 839.3     |
| 11        | -0.1766   | 0.09758   | 59.31     | 0.03966   | 156.2     |
| 12        | -0.1761   | 0.09311   | 

MLPRegressor(alpha=0.0813060031937371, hidden_layer_sizes=(94,),
             learning_rate_init=0.07055621355576995, max_iter=961)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 0.1093364198492914,
  'mse': 0.03137962485868217,
  'rmse': 0.17714295034994243,
  'r2': 0.009968999119732369,
  'adj_r2': -0.06471006344666708},
 {'model': 'Random Forest',
  'mae': 0.1089646914744536,
  'mse': 0.03148077494974815,
  'rmse': 0.17742822478328568,
  'r2': 0.006777701379616752,
  'adj_r2': -0.06814208407666222},
 {'model': 'Neural Network',
  'mae': 0.10895347391711734,
  'mse': 0.031694704383603584,
  'rmse': 0.17803006595405055,
  'r2': 2.820158567495845e-05,
  'adj_r2': -0.07540070562230206},
 {'model': 'Support Vector Machine',
  'mae': 0.10757936537701356,
  'mse': 0.03172211775653611,
  'rmse': 0.17810704016555917,
  'r2': -0.0008366936820063398,
  'adj_r2': -0.07633084083474384}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison Conditie',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 0.093769038068442,
  'mse': 0.018923917178484576,
  'rmse': 0.13756422928394058,
  'r2': -0.021818228880130075,
  'adj_r2': -0.4214119496712423},
 {'model': 'Random Forest',
  'mae': 0.09349362222114112,
  'mse': 0.01890203479608603,
  'rmse': 0.13748467113131568,
  'r2': -0.020636664988507025,
  'adj_r2': -0.41976832168792333},
 {'model': 'Neural Network',
  'mae': 0.09273273712628642,
  'mse': 0.018416571876215072,
  'rmse': 0.13570767066092865,
  'r2': 0.005576452321766401,
  'adj_r2': -0.3833042646473752},
 {'model': 'Support Vector Machine',
  'mae': 0.09170750747861986,
  'mse': 0.01829977159101541,
  'rmse': 0.13527664835815312,
  'r2': 0.011883215315376727,
  'adj_r2': -0.3745311697568223}]

In [18]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        text=metric_scores[metric],  # Display the values on the bars
        textposition='auto',  # Position the text inside the bars
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison Conditie',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 0.09220708558461792
Mean Squared Error: 0.018520778761338827
Root Mean Squared Error: 0.13609106789697414
R2 Score: -5.031584626036256e-05
Adjusted R2 Score: -0.3911314449481498


Save best model as Pickle

In [17]:
import pickle

# Save model
file_path = 'Saved Models/Conditie.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_Conditie.txt', 'w') as file:
    file.write(f'{best_features_mlr}')